### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

### Data Understanding

In [2]:
df = pd.read_csv("persona.csv")

In [3]:
print(df.shape)

(5000, 5)


In [4]:
print(df.head())

   PRICE   SOURCE   SEX COUNTRY  AGE
0     39  android  male     bra   17
1     39  android  male     bra   17
2     49  android  male     bra   17
3     29  android  male     tur   17
4     49  android  male     tur   17


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB
None


### Rule Creation

In [6]:
df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"PRICE": "sum"})

PRICE
COUNTRY SOURCE  SEX    AGE       
bra     android female 15    1355
                       16    1294
                       17     642
                       18    1387
                       19    1021
...                           ...
usa     ios     male   42     242
                       50     156
                       53      68
                       55      29
                       59     186

[348 rows x 1 columns]

In [7]:
agg_df =  (df.groupby(["COUNTRY","SOURCE","SEX","AGE",]).agg({"PRICE": "sum"} )).sort_values("PRICE",ascending=False)
agg_df

PRICE
COUNTRY SOURCE  SEX    AGE       
usa     android male   15    3917
bra     android male   19    2606
usa     ios     male   15    2496
        android female 20    2190
deu     ios     female 16    2169
...                           ...
tur     ios     male   47      19
can     android female 27      19
tur     android male   21      19
usa     ios     female 30      19
deu     android male   26       9

[348 rows x 1 columns]

In [8]:
agg_df = agg_df.reset_index()
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,usa,android,male,15,3917
1,bra,android,male,19,2606
2,usa,ios,male,15,2496
3,usa,android,female,20,2190
4,deu,ios,female,16,2169
...,...,...,...,...,...
343,tur,ios,male,47,19
344,can,android,female,27,19
345,tur,android,male,21,19
346,usa,ios,female,30,19


In [9]:
df["AGE"].nunique()

46

In [10]:
agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], [0, 18, 25, 35, 45, 67], labels=['0_18', '19_25', '26_35', '36-45', '46_67'])
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,usa,android,male,15,3917,0_18
1,bra,android,male,19,2606,19_25
2,usa,ios,male,15,2496,0_18
3,usa,android,female,20,2190,19_25
4,deu,ios,female,16,2169,0_18
...,...,...,...,...,...,...
343,tur,ios,male,47,19,46_67
344,can,android,female,27,19,26_35
345,tur,android,male,21,19,19_25
346,usa,ios,female,30,19,26_35


In [11]:
agg_df["customers_level_based"] = [col[0].upper() + "_" + col[1].upper() + "_" + col[2].upper() + "_" + col[5].upper() for col in agg_df.values]
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
0,usa,android,male,15,3917,0_18,USA_ANDROID_MALE_0_18
1,bra,android,male,19,2606,19_25,BRA_ANDROID_MALE_19_25
2,usa,ios,male,15,2496,0_18,USA_IOS_MALE_0_18
3,usa,android,female,20,2190,19_25,USA_ANDROID_FEMALE_19_25
4,deu,ios,female,16,2169,0_18,DEU_IOS_FEMALE_0_18


In [12]:
agg_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"})

In [13]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4 , labels = ["D","C","B","A"])
agg_df

,PRICE,SEGMENT
customers_level_based,,
BRA_ANDROID_FEMALE_0_18,1169.500000,A
BRA_ANDROID_FEMALE_19_25,1063.428571,A
BRA_ANDROID_FEMALE_26_35,327.750000,B
BRA_ANDROID_FEMALE_36-45,298.166667,C
BRA_ANDROID_FEMALE_46_67,185.000000,D
...,...,...
USA_IOS_MALE_0_18,1566.500000,A
USA_IOS_MALE_19_25,867.750000,A
USA_IOS_MALE_26_35,378.125000,B


In [14]:
agg_df.groupby("SEGMENT").agg({"PRICE": ["mean","max","sum"]})

PRICE                      
               mean     max           sum
SEGMENT                                  
D        119.015432   185.0   3213.416667
C        247.767333   311.0   6194.183333
B        371.528526   472.0   9659.741667
A        917.976801  1992.5  23867.396825

In [15]:
agg_df = agg_df.reset_index()
agg_df

,customers_level_based,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,1169.500000,A
1,BRA_ANDROID_FEMALE_19_25,1063.428571,A
2,BRA_ANDROID_FEMALE_26_35,327.750000,B
3,BRA_ANDROID_FEMALE_36-45,298.166667,C
4,BRA_ANDROID_FEMALE_46_67,185.000000,D
...,...,...,...
99,USA_IOS_MALE_0_18,1566.500000,A
100,USA_IOS_MALE_19_25,867.750000,A
101,USA_IOS_MALE_26_35,378.125000,B
102,USA_IOS_MALE_36-45,229.000000,C


### For Example

In [16]:
# Which segment does a 33 year old Turkish woman using ANDROID belong to? 
#How much income is expected on average?

user_1  = "TUR_ANDROID_FEMALE_26_35"
agg_df[agg_df["customers_level_based"] ==  user_1 ]  # 413.75  B

,customers_level_based,PRICE,SEGMENT
68,TUR_ANDROID_FEMALE_26_35,413.75,B
